# Gaussian Mixture Models (GMM)

### Description

The Mixture of Gaussian (MoG) or Gaussian Mixture Models (GMM) is a prototypical exmaple of a model where learning is suited to the Expectation Maximization algorithm (EM-Algorithm). The date are described as a weighted sum of K normal distributions
\begin{align}
Pr(X_i|\theta) = \sum_{k=1}^{K} \lambda_k \text{Norm}_{X_i} [\mu_k, \Sigma_k],
\end{align}
where $\mu_{1...K}$ and $\Sigma_{1...K}$ are the means and the covariances of the normal distributions and $\lambda_{1...K}$ are positive valued weights that sum to one.

In the expectation step :
\begin{align}
r_{ik} = \dfrac{\lambda_k \text{Norm}_{X_i} [\mu_k, \Sigma_k]}{\sum_{j=1}^{K} \lambda_j \text{Norm}_{X_i} [\mu_j, \Sigma_j]}
\end{align}

In the Maximization step :
\begin{align}
\begin{cases}
\lambda_k^{[t+1]} = \dfrac{\sum_{i=1}^{K}r_{ik}}{\sum_{j=1}^{K} \sum_{i=1}^{I}r_{ij}},
\\\mu_{k}^{[t+1]} = \dfrac{\sum_{i=1}^{K}r_{ik}X_i}{\sum_{i=1}^{I}r_{ij}},
\\\Sigma_{k}^{[t+1]} = \dfrac{\sum_{i=1}^{K}r_{ik}(X_i - \mu_{k}^{[t+1]})(X_i - \mu_{k}^{[t+1]})^T}{\sum_{i=1}^{I}r_{ij}}.
\end{cases}
\end{align}

In [84]:
import numpy as np
from scipy.stats import multivariate_normal
import cv2
import matplotlib.pyplot as plt
from natsort import natsorted
from glob import glob
import skimage
from skimage import img_as_float32
import imageio as iio
import json
import random
import math

In [27]:
class GaussianMixtureModel:
    def __init__(self, k, max_iter=5):
        self.k = k
        self.max_iter = int(max_iter)

    def initialize(self, X):
        # returns the (r,c) value of the numpy array of X
        self.shape = X.shape 
        # n has the number of rows while m has the number of columns of dataset X
        self.n, self.m = self.shape 
        

        # initial weights given to each cluster are stored in phi or P(Ci=j)
        self.Lambda = np.full(shape=self.k, fill_value=1/self.k) 

        # initial weights given to each data point wrt to each cluster or P(Xi/Ci=j)
        self.weights = np.full(shape=self.shape, fill_value=1/self.k)
        
        # dataset is divided randomly into k parts of unequal sizes
        random_row = np.random.randint(low=0, high=self.n, size=self.k)

        # initial value of mean of k Gaussians
        self.mu = [ X[row_index,:] for row_index in random_row ] 

        # initial value of covariance matrix of k Gaussians
        self.sigma = [ np.cov(X.T) for _ in range(self.k) ] 
        # theta =(mu_1,sigma_1,mu_2,simga_2......mu_k,sigma_k)

    # E-Step: update weights and phi holding mu and sigma constant
    def e_step(self, X):
        # updated weights or P(Xi/Ci=j)
        self.weights = self.predict_proba(X)
        # mean of sum of probability of all data points wrt to one cluster is new updated probability of cluster k 
        # or lambda_k
        self.Lambda = self.weights.mean(axis=0)

    # M-Step: update meu and sigma holding phi and weights constant
    def m_step(self, X):
        for i in range(self.k):
            weight = self.weights[:, [i]]
            total_weight = weight.sum()

            self.mu[i] = (X * weight).sum(axis=0) / total_weight
            self.sigma[i] = np.cov(X.T,aweights=(weight/total_weight).flatten(), bias=True)

    # responsible for clustering the data points correctly
    def fit(self, X):
        # initialise parameters like weights: responsability, lambda, mu, sigma of all Gaussians in dataset X
        self.initialize(X)
        plt.figure(figsize=(16, 25))
        for iteration in range(self.max_iter):
            permutation = np.array([mode(Image.target[gmm.predict(X) == i]).mode.item() for i in range(gmm.k)])
            permuted_prediction = permutation[gmm.predict(X)]
            print('\nThe accuracy of the permuted prediction against target before iteration ',iteration+1,end="")
            print(': ',np.mean(Image.target == permuted_prediction))
            print('\n')
            confusion_matrix(Image.target, permuted_prediction)
            plt.title(' Iteration Cluster')
            plt.subplot(5,3,iteration+1)
            clusters=permuted_prediction
            plt.xlabel(Image.feature_names[0])
            plt.ylabel(Image.feature_names[1])
            plt.scatter(jitter(X[:, 0]), jitter(X[:, 1]), c=clusters, cmap=plt.cm.get_cmap('brg'),marker='.')
            plt.grid()
            plt.tight_layout()
            # iterate to update the value of P(Xi/Ci=j) and (phi)k
            self.e_step(X)
            # iterate to update the value of meu and sigma as the clusters shift
            self.m_step(X)
            
    # predicts probability of each data point wrt each cluster
    def predict_proba(self, X):
        # Creates a n*k matrix denoting probability of each point wrt each cluster 
        likelihood = np.zeros( (self.n, self.k) ) 
        for i in range(self.k):
            distribution = multivariate_normal(mean=self.mu[i],cov=self.sigma[i])
            # pdf : probability denisty function
            likelihood[:,i] = distribution.pdf(X) 

        numerator = likelihood * self.phi
        denominator = numerator.sum(axis=1)[:, np.newaxis]
        weights = numerator / denominator
        return weights
    
    # predict function 
    def predict(self, X):
        weights = self.predict_proba(X)
        # datapoint belongs to cluster with maximum probability
        # returns this value
        return np.argmax(weights, axis=1)

In [271]:
path_pairs = list(zip(
    natsorted(glob('./puzzle_corners_1024x768/images-1024x768/*.png')),
    natsorted(glob('./puzzle_corners_1024x768/masks-1024x768/*.png')),
))

imgs = np.array([img_as_float32(iio.imread(ipath)) for ipath, _ in path_pairs])
msks = np.array([img_as_float32(iio.imread(mpath)) for _, mpath in path_pairs])

C:\Users\abdel\anaconda3\envs\rstudio\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  
C:\Users\abdel\anaconda3\envs\rstudio\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  import sys


In [273]:
imgs.shape

(48, 768, 1024, 3)

Helper function to divide the data into training (70%), validation (15%) and test (15%) sets.

In [293]:
X = imgs
Y = msks

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=1)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.17, random_state=1)

In [294]:
x_train.shape


(33, 768, 1024, 3)

In [295]:
x_val.shape

(7, 768, 1024, 3)

In [296]:
y_train.shape

(33, 768, 1024)

In [297]:
y_val.shape

(7, 768, 1024)

In [5]:
np.random.seed(42)
gmm = GaussianMixtureModel(k=3, max_iter=14)
gmm.fit(X)

NameError: name 'X' is not defined